## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [5]:
import numpy as np
import pandas as pd
from string import ascii_uppercase

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [18]:
%%time

A = np.random.randint(0, 1000, size=1_000_000)
f = lambda x: x + 100
vectorize = np.vectorize(f)
B = vectorize(A)
print(f"{A[0] = }  | {B[0] = }")
print(f"\n{A.mean() = }\n{B.mean() = }\n")

A[0] = 374  | B[0] = 474

A.mean() = 499.498606
B.mean() = 599.498606

CPU times: total: 453 ms
Wall time: 505 ms


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [19]:
%%time
X = np.random.randint(0, 5000, size=2_000_000)
df = pd.DataFrame({
    'col1': X,
    'col2': np.vectorize(lambda x: x // 2)(X),
    'col3': np.vectorize(lambda x: x * 1.5)(X),
    'col4': np.vectorize(lambda x: x - 100)(X)
})

alphabet = ascii_uppercase
lindex = np.random.randint(0, len(alphabet), size=2_000_000)
key = np.vectorize(lambda x: alphabet[x])(lindex)

df["key"] = key
print(key[:5])
result = key[:5]

['Q' 'T' 'E' 'A' 'W']
CPU times: total: 4.17 s
Wall time: 4.34 s


In [23]:
df.loc[df["key"].isin(result)]

,col1,col2,col3,col4,key
0,2712,1356,4068.0,2612,Q
1,1856,928,2784.0,1756,T
2,3754,1877,5631.0,3654,E
3,1498,749,2247.0,1398,A
4,1682,841,2523.0,1582,W
...,...,...,...,...,...
1999946,1632,816,2448.0,1532,A
1999967,4851,2425,7276.5,4751,E
1999971,4128,2064,6192.0,4028,W
1999982,2666,1333,3999.0,2566,T


## Лабораторная работа 3

In [24]:
#!pip install line_profiler

                                              0.0/84.7 kB ? eta -:--:--
     ------------------                     41.0/84.7 kB 991.0 kB/s eta 0:00:01
     --------------------------------------   81.9/84.7 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 84.7/84.7 kB 792.3 kB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [2]:
recipes = pd.read_csv("../data sources/recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')
recipes.head()

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [8]:
reviews = pd.read_csv("../data sources/reviews_sample.csv", sep=",", parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

reviews.head()

,user_id,recipe_id,date,rating,review
id,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


# A. С использованием метода `DataFrame.iterrows` исходной таблицы;

In [65]:
def get_average_for_iter(reviews):
    total = count = 0

    for index, row in reviews.iterrows():
        total += row["rating"]
        count += 1

    return total / count

average_iter = get_average_for_iter(reviews)

In [66]:
%%timeit
get_average_for_iter(reviews)

14.2 s ± 1.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

# Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

In [68]:
reviews_for_2010 = reviews.loc[reviews["date"].dt.year == 2010]

In [77]:
def get_average_for_2010_iter(reviews):
    total = count = 0

    for index, row in reviews.iterrows():
        total += row["rating"]
        count += 1

    return total / count   

average_iter_2010 = get_average_for_2010_iter(reviews_for_2010)

In [70]:
%%timeit
get_average_for_2010_iter(reviews_for_2010)

1.67 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# В. С использованием метода `Series.mean`.


In [71]:
%%timeit 

reviews.rating.mean()

350 µs ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [78]:
# Проверка

average_iter == reviews.rating.mean(), average_iter_2010 == reviews.loc[reviews["date"].dt.year == 2010]["rating"].mean()

(True, True)

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [79]:
#%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [87]:
%lprun -f get_average_for_iter get_average_for_iter(reviews)

Timer unit: 1e-07 s

Total time: 54.7848 s
File: C:\Users\cvrsd\AppData\Local\Temp\ipykernel_9628\746794879.py
Function: get_average_for_iter at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_average_for_iter(reviews):
     2         1         18.0     18.0      0.0      total = count = 0
     3                                           
     4    126696  474930481.0   3748.6     86.7      for index, row in reviews.iterrows():
     5    126696   71334028.0    563.0     13.0          total += row["rating"]
     6    126696    1583866.0     12.5      0.3          count += 1
     7                                           
     8         1         24.0     24.0      0.0      return total / count

In [89]:
%lprun -f get_average_for_2010_iter get_average_for_2010_iter(reviews_for_2010)

Timer unit: 1e-07 s

Total time: 5.04593 s
File: C:\Users\cvrsd\AppData\Local\Temp\ipykernel_9628\1764220426.py
Function: get_average_for_2010_iter at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_average_for_2010_iter(reviews):
     2         1         30.0     30.0      0.0      total = count = 0
     3                                           
     4     12094   43788691.0   3620.7     86.8      for index, row in reviews_for_2010.iterrows():
     5     12094    6525443.0    539.6     12.9          total += row["rating"]
     6     12094     145157.0     12.0      0.3          count += 1
     7                                           
     8         1         20.0     20.0      0.0      return total / count

In [90]:
def get_average(reviews):
    return reviews.rating.mean()

In [91]:
%lprun -f get_average get_average(reviews)

Timer unit: 1e-07 s

Total time: 0.0012072 s
File: C:\Users\cvrsd\AppData\Local\Temp\ipykernel_9628\3641444888.py
Function: get_average at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_average(reviews):
     2         1      12072.0  12072.0    100.0      return reviews.rating.mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [34]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [36]:
%timeit get_word_reviews_count(reviews)

8.24 s ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 20.423 s
File: /var/folders/8q/wkt9c6yd5mb6ddqdsbdz0c5h0000gn/T/ipykernel_15419/2826575548.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1       3000.0   3000.0      0.0      word_reviews = {}
     3    126679 5231373000.0  41296.3     25.6      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 1213175000.0   9576.8      5.9          recipe_id, review = row['recipe_id'], row['review']
     5    126679  325181000.0   2567.0      1.6          words = review.split(' ')
     6   6792010  819126000.0    120.6      4.0          for word in words:
     7   6617066 1383331000.0    209.1      6.8              if word not in word_reviews:
     8    174944   42027000.0    240.2      0.2                  word_reviews[word] = []
     9   6792010 1926351000.0    283.6      9.4              word_reviews[word].append(recipe_id)
    10                                               
    11         1       1000.0   1000.0      0.0      word_reviews_count = {}
    12    126679 5026525000.0  39679.2     24.6      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679  719650000.0   5680.9      3.5          review = row['review']
    14    126679  322284000.0   2544.1      1.6          words = review.split(' ')
    15   6792010  878885000.0    129.4      4.3          for word in words:
    16   6792010 2535064000.0    373.2     12.4              word_reviews_count[word] = len(word_reviews[word])
    17         1       1000.0   1000.0      0.0      return word_reviews_count

In [39]:
def get_word_reviews_count_optimized(df):
    
    word_reviews = {}

    for _, row in df.dropna(subset=['review']).iterrows():

        for word in row['review'].split(' '):
            if word not in word_reviews:
                word_reviews[word] = 0
            word_reviews[word] += 1
    
    return word_reviews

In [40]:
%timeit get_word_reviews_count_optimized(reviews)

3.83 s ± 43.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

Timer unit: 1e-09 s

Total time: 10.1141 s
File: /var/folders/8q/wkt9c6yd5mb6ddqdsbdz0c5h0000gn/T/ipykernel_15419/1720791363.py
Function: get_word_reviews_count_optimized at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count_optimized(df):
     2                                               
     3         1       3000.0   3000.0      0.0      word_reviews = {}
     4                                           
     5    126679 5023973000.0  39659.1     49.7      for _, row in df.dropna(subset=['review']).iterrows():
     6                                           
     7   6792010 1781923000.0    262.4     17.6          for word in row['review'].split(' '):
     8   6617066 1326627000.0    200.5     13.1              if word not in word_reviews:
     9    174944   37842000.0    216.3      0.4                  word_reviews[word] = 0
    10   6792010 1943722000.0    286.2     19.2              word_reviews[word] += 1
    11                                               
    12         1          0.0      0.0      0.0      return word_reviews

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [9]:
sample_recipe_id_from_reviews = reviews.sample(10) # Выберем 10 строк из reviews, по ним будем считать MAPE
sample_recipe_id_from_reviews

,user_id,recipe_id,date,rating,review
id,,,,,
208189,2000045537,488899,2015-03-23,5,This is so good we love this :) thank you :)
676963,32772,5018,2003-06-05,5,My in-laws are up from Florida and I was calle...
520424,2710455,495291,2013-02-24,5,I Love and Cheese. Add ranch dressing seasonin...
1093932,1316222,319219,2009-07-06,5,Just made this for dinner and the whole family...
376099,154287,60238,2008-07-01,3,It was just Ok for us. I used garlic flavore...
1102708,938649,234344,2010-09-14,5,Made this for my mother as a more appealing al...
246551,270514,115110,2007-03-29,4,I agree that there was a little bit too much s...
626586,207668,58367,2007-04-03,5,I was brave and took this torte to our Seder. ...
1074204,303162,135350,2007-03-08,4,"Yes, total comfort food... my family loves it ..."


#    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`


In [10]:
def mape_noNumpy_noNumba(df, recipe_id):
    ratings = df[df['recipe_id'] == recipe_id]['rating']
    mean_rating = ratings.mean()
    mape = 0
    for rating in ratings:
        mape += abs(rating - mean_rating) / mean_rating
    mape /= len(ratings)
    return mape * 100

[print(f"{'MAPE для рецепта с id=':>20}{recipe_id:<6}: {round(mape_noNumpy_noNumba(reviews, recipe_id), 2):>5}%") for recipe_id in sample_recipe_id_from_reviews["recipe_id"].values]

MAPE для рецепта с id=488899:   0.0%
MAPE для рецепта с id=5018  : 15.24%
MAPE для рецепта с id=495291: 10.77%
MAPE для рецепта с id=319219: 13.33%
MAPE для рецепта с id=60238 : 11.66%
MAPE для рецепта с id=234344: 47.88%
MAPE для рецепта с id=115110: 23.95%
MAPE для рецепта с id=58367 :   0.0%
MAPE для рецепта с id=135350: 23.97%
MAPE для рецепта с id=135350: 23.97%


[None, None, None, None, None, None, None, None, None, None]

In [11]:
# Проверка
reviews[reviews['recipe_id'] == 488899]['rating'] # MAPE рецепта 488899 равен 0, т.к. имеется всего лишь одна оценка, разброса нет.

id
208189    5
Name: rating, dtype: int64

In [12]:
# Проверка
reviews[reviews['recipe_id'] == 234344]['rating'] # MAPE рецепта 42603 имеет намного больше оценок с разными значениями, поэтому MAPE > 0

id
1102713    5
1102715    1
1102740    5
1102726    4
1102742    5
          ..
1102743    2
1102759    0
1102731    3
1102708    5
1102738    5
Name: rating, Length: 71, dtype: int64

In [13]:
# Проверка
reviews[reviews['recipe_id'] == 58367]['rating'] # MAPE так же равен 0, разброса по оценкам нет

id
626586    5
626585    5
Name: rating, dtype: int64

In [38]:
%%timeit

mape_noNumpy_noNumba(reviews, 234344)

602 µs ± 187 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#        2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`



In [17]:
sample_recipe_id_from_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 208189 to 1074578
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    10 non-null     int64         
 1   recipe_id  10 non-null     int64         
 2   date       10 non-null     datetime64[ns]
 3   rating     10 non-null     int64         
 4   review     10 non-null     object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 480.0+ bytes


In [34]:
import numba as nb


@nb.njit
def mape_noNumpy_withNumba(ratings, recipe_id):
    
    mean_rating = ratings.mean()
    mape = 0
    
    for rating in ratings:
        mape += abs(rating - mean_rating) / mean_rating
        
    mape /= len(ratings)
    return mape * 100



for recipe_id in sample_recipe_id_from_reviews["recipe_id"].values:
    ratings = reviews[reviews['recipe_id'] == recipe_id]['rating'].values
    
    print(f"{'MAPE для рецепта с id=':>20}{recipe_id:<6}: {round(mape_noNumpy_withNumba(ratings, recipe_id), 2):>5}%")

# Вроде как совпадают

MAPE для рецепта с id=488899:   0.0%
MAPE для рецепта с id=5018  : 15.24%
MAPE для рецепта с id=495291: 10.77%
MAPE для рецепта с id=319219: 13.33%
MAPE для рецепта с id=60238 : 11.66%
MAPE для рецепта с id=234344: 47.88%
MAPE для рецепта с id=115110: 23.95%
MAPE для рецепта с id=58367 :   0.0%
MAPE для рецепта с id=135350: 23.97%
MAPE для рецепта с id=135350: 23.97%


In [36]:
ratings = reviews[reviews['recipe_id'] == 234344]['rating'].values

ratings

array([5, 1, 5, 4, 5, 3, 5, 0, 0, 5, 5, 5, 5, 5, 5, 2, 0, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 0, 0, 5, 5, 5, 0, 5, 5, 0, 5, 5, 0, 4, 5, 5, 4, 5, 5,
       5, 0, 5, 5, 5, 5, 5, 4, 0, 4, 5, 5, 5, 5, 5, 0, 5, 0, 5, 0, 0, 4,
       2, 0, 3, 5, 5], dtype=int64)

In [37]:
%%timeit

mape_noNumpy_withNumba(ratings, 234344)

620 ns ± 123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


#    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`


In [49]:
def mape_withNumpy_noNumba(df, recipe_id):
    ratings = df[df['recipe_id'] == recipe_id]
    
    mean_rating = np.mean(ratings['rating'])
    
    val_rating = np.abs(ratings['rating'] - mean_rating) / mean_rating

    if val_rating.any():
        return np.sum(val_rating) / val_rating.shape[0] * 100
        
    return 0

[print(f"{'MAPE для рецепта с id=':>20}{recipe_id:<6}: {round(mape_noNumpy_noNumba(reviews, recipe_id), 2):>5}%") for recipe_id in sample_recipe_id_from_reviews["recipe_id"].values]

# Вроде как совпадают

MAPE для рецепта с id=488899:   0.0%
MAPE для рецепта с id=5018  : 15.24%
MAPE для рецепта с id=495291: 10.77%
MAPE для рецепта с id=319219: 13.33%
MAPE для рецепта с id=60238 : 11.66%
MAPE для рецепта с id=234344: 47.88%
MAPE для рецепта с id=115110: 23.95%
MAPE для рецепта с id=58367 :   0.0%
MAPE для рецепта с id=135350: 23.97%
MAPE для рецепта с id=135350: 23.97%


[None, None, None, None, None, None, None, None, None, None]

In [50]:
%%timeit

mape_withNumpy_noNumba(reviews, 234344)

1.7 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`


In [30]:
@nb.njit
def mape_withNumpy_withNumba(ratings, recipe_id):
    
    mean_rating = ratings.mean()
    
    val_rating = np.abs(ratings - mean_rating) / mean_rating
            
    if val_rating.any():
        return np.sum(val_rating) / val_rating.shape[0] * 100
        
    return 0



for recipe_id in sample_recipe_id_from_reviews["recipe_id"].values:
    ratings = reviews[reviews['recipe_id'] == recipe_id]['rating'].values
    
    print(f"{'MAPE для рецепта с id=':>20}{recipe_id:<6}: {round(mape_withNumpy_withNumba(ratings, recipe_id), 2):>5}%")


MAPE для рецепта с id=488899:   0.0%
MAPE для рецепта с id=5018  : 15.24%
MAPE для рецепта с id=495291: 10.77%
MAPE для рецепта с id=319219: 13.33%
MAPE для рецепта с id=60238 : 11.66%
MAPE для рецепта с id=234344: 47.88%
MAPE для рецепта с id=115110: 23.95%
MAPE для рецепта с id=58367 :   0.0%
MAPE для рецепта с id=135350: 23.97%
MAPE для рецепта с id=135350: 23.97%


In [51]:
ratings = reviews[reviews['recipe_id'] == 234344]['rating'].values

In [52]:
%%timeit

mape_withNumpy_withNumba(ratings, 234344)

1.23 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
